1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지못한다.
    - 문서가 길면(인풋이 길면) 답변 생성이 오래 걸린다.
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [6]:
#%pip install --upgrade pip
#%pip install langchain langchain-community langchain-upstage langchain-text-splitters python-dotenv docx2txt

Note: you may need to restart the kernel to use updated packages.
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3990 sha256=b918803a625e95f99165ece6f203e6e2aadddce7c361fb3cb2beefbfdf0f49f1
  Stored in directory: /Users/dot/Library/Caches/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
Note: you may need to restart the kernel to use updated packages.


In [70]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter)

In [58]:
document_list[52].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제3항 각 호 외의 부분 단서가 적용되는 배당소득금액이 합산되어 있는 경우에는 같은 항 각 호 외의 부분 단서에 따라 해당 과세기간의 총수입금액에 더한 금액에 해당하는 금액을 종합소득 산출세액에서 공제한다. <개정 2009. 12. 31.>\n\n② 제1항에 따른 공제를 “배당세액공제”라 한다.<개정 2009. 12. 31.>\n\n③ 삭제<2003. 12. 30.>\n\n④ 제1항을 적용할 때 배당세액공제의 대상이 되는 배당소득금액은 제14조제2항의 종합소득과세표준에 포함된 배당소득금액으로서 이자소득등의 종합과세기준금액을 초과하는 것으로 한다.<개정 2009. 12. 31.>\n\n⑤ 삭제<2006. 12. 30.>'

In [71]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [ ]:
"""
%pip install --upgrade --quiet \
    langchain-pinecone\
        langchain-upstage\
            langchain
"""

Note: you may need to restart the kernel to use updated packages.


In [72]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name = 'tax-table-index'

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [82]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=3)

In [62]:
#retrieved_docs
from langchain_upstage import ChatUpstage
llm = ChatUpstage()

In [24]:
'''
prompt = f"""
[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""
'''

'\nprompt = f"""\n[Identity]\n- 당신은 최고의 한국 소득세 전문가입니다.\n- [Context]를 참고해서 사용자의 질문에 답변해주세요.\n\n[Context]\n{retrieved_docs}\n\nQuestion: {query}\n"""\n'

In [25]:
#ai_message = llm.invoke(prompt)
#ai_message.content

In [ ]:
#%pip install -U langchain langchainhub

In [74]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/dot/.pyenv/versions/3.10.16/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [75]:
from langchain.chains import RetrievalQA

retriever=database.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [76]:
retriever.invoke(query)

[Document(id='801c8f6f-bff6-4395-9c6e-6155e6f111bd', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

In [80]:
ai_message = qa_chain({"query": query})

In [81]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요? 라는 질문에 539만6천원이라는 답변을 주셨습니다. 5000만원이면, 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)로 계산해야 되는 것 아닌가요?',
 'result': '5천만원이면, 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)가 맞습니다. 다만, 누진세 구조이기 때문에 초과분에 대해서만 과세하는 것은 아니고, 이미 15% 세율 구간에서 세금을 납부한 부분이 있으므로, 전체 구간을 적용하여 계산하고 이미 납부한 세금은 공제하는 방식으로 계산합니다. 이러한 방식으로 계산할 경우 539만 6천원이 나오는 것입니다.'}